## downsample

In [ ]:
import rioxarray as rxr

src = "gebco_2025_sub_ice_topo_geotiff/gebco_2025_sub_ice_n90.0_s0.0_w-180.0_e-90.0.tif"   # <-- change to your filename


da = rxr.open_rasterio(src, masked=True)

subset = da.rio.clip_box(
    minx=-175,
    miny=15,
    maxx=-120,
    maxy=65
)

subset.rio.to_raster("gebco_2025_crop.tif")
print("saved cropped file ✅")


## grid

In [ ]:
import rioxarray as rxr
import pandas as pd
import numpy as np

da = rxr.open_rasterio("gebco_2025_crop.tif", masked=True)

step = 40 

small = da.isel(
    y=slice(None, None, step),
    x=slice(None, None, step)
)

arr = small.values.squeeze()         
lats = small.y.values
lons = small.x.values

lon_grid, lat_grid = np.meshgrid(lons, lats)

df = pd.DataFrame({
    "lon": lon_grid.ravel(),
    "lat": lat_grid.ravel(),
    "depth": arr.ravel()
})

df = df.dropna(subset=["depth"])

df.to_csv("bathy_grid_points.csv", index=False)
print("saved bathy_grid_points.csv", df.shape)
